In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
sys.path.append('../src')
import escrapeos as esc
import json


# Escrapeo Nombre trastronos CIE-10

[wiki](https://es.wikipedia.org/wiki/Anexo:CIE-10_Cap%C3%ADtulo_V:_Trastornos_mentales_y_del_comportamiento)

In [2]:
url = "https://es.wikipedia.org/wiki/Anexo:CIE-10_Cap%C3%ADtulo_V:_Trastornos_mentales_y_del_comportamiento"
elemento = 'h2'
# html = requests.get(url)
# soup = BeautifulSoup(html.content,"html.parser")

In [3]:
capitulo = esc.capitulos(url,elemento)

In [4]:
capitulo.sample(3)

,cap,title
10,F99,Trastornos mentales sin especificar
6,F60-69,Trastornos de la personalidad y del comportami...
1,F10-F19,Trastornos mentales y de comportamiento debido...


In [5]:
elementodos = 'li'

In [6]:
diagnostic = esc.diagnostics(url,elementodos)

In [7]:
diagnostic.sample(3)

,code,diag
283,(F98.2),Trastorno de la alimentación de la infancia y ...
78,F30.2,"Episodio maníaco, grave, con síntomas psicóticos"
175,(F51.5),Pesadillas


### limpieaza dataframe diag

In [8]:
sin_p = {r"[(]":"",r"[)]":""}#quitamos los paréntesis 
diagnostic.code = diagnostic.code.replace(sin_p,regex=True) 


In [9]:
diagnostic.sample(3)

,code,diag
232,F80,Trastornos específicos del lenguaje y del habl...
95,F31.4,"Trastorno bipolar, episodio actual depresivo, ..."
48,F20.4,Depresión post-esquizofrénica


In [10]:
falta = {'code':'F65','diag':'Trastornos de la preferencia sexual'}

In [11]:
diagnostic = diagnostic.append(falta, ignore_index=True)

In [14]:
diagnostic.tail(3)

,code,diag
287,F98.6,Desorden lingüístico
288,F99,Trastorno mental no especificado en otra parte
289,F65,Trastornos de la preferencia sexual


In [15]:
#nos quedamos solo con los subcodigos que sean principales (F00-F99)
diagnostic = diagnostic[diagnostic["code"].str.match(r"(^F..$)")]
len(diagnostic)

77

In [16]:
for i,r in diagnostic.iterrows():
    if "\n" in r.diag:
        r.diag = r.diag.split("\n")[0]

In [17]:
diagnostic.sample(3)

,code,diag
260,F91,Trastornos de conducta
228,F72,Retraso mental severo
196,F62,"Cambios de personalidad duraderos, no atribuib..."


In [18]:
#añadimos el capítulo correpondiente a cada diagnóstico de diagnostic

In [19]:
correspond = {'F00-F09': ["F00","F01","F02","F03","F04","F05","F06","F07","F08","F09"],
'F10-F19': ["F10","F11","F12","F13","F14","F15","F16","F17","F18","F19"],
'F20-29' : ["F20","F21","F22","F23","F24","F25","F26","F27","F28","F29"],
'F30-39' : ["F30","F31","F32","F33","F34","F35","F36","F37","F38","F39"],
'F40-49': ["F40","F41","F42","F43","F44","F45","F46","F47","F48","F49"],
'F50-59': ["F50","F51","F52","F53","F54","F55","F56","F57","F58","F59"],
'F60-69': ["F60","F61","F62","F63","F64","F65","F66","F67","F68","F69"],
'F70-79':["F70","F71","F72","F73","F74","F75","F76","F77","F78","F79"],
'F80-89' : ["F80","F81","F82","F83","F84","F85","F86","F87","F88","F89"],
'F90-F98':["F90","F91","F92","F93","F94","F95","F96","F97","F98"],
'F99':["F99"]}

In [20]:
dicc_corr = {}
for key,value in correspond.items():
    for i in range(len(value)):
        dicc_corr[value[i]] = key

In [21]:
diagnostic['cap']=diagnostic.code.map(dicc_corr)

<ipython-input-21-f0f5854a54f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostic['cap']=diagnostic.code.map(dicc_corr)


In [22]:
diagnostic = diagnostic.merge(capitulo, how='right')

In [23]:
diagnostic.sample(10)

,code,diag,cap,title
70,F91,Trastornos de conducta,F90-F98,Trastornos emocionales y del comportamiento qu...
15,F15,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
43,F52,Disfunción sexual no ocasionada por trastornos...,F50-59,Síndromes del comportamiento asociados con alt...
24,F24,Trastorno de ideas delirantes inducidas,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."
14,F14,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
10,F10,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
74,F95,Tics,F90-F98,Trastornos emocionales y del comportamiento qu...
68,F89,Trastornos del desarrollo psicológico sin espe...,F80-89,Trastornos del desarrollo psicológico
51,F64,Trastornos de identidad de género,F60-69,Trastornos de la personalidad y del comportami...
55,F65,Trastornos de la preferencia sexual,F60-69,Trastornos de la personalidad y del comportami...


In [24]:
diagnostic.to_csv("../data/diagnos.csv", index=False)

#  Fases lunares

In [4]:
url_ = 'https://calendarios.ideal.es/calendario-lunar/fases-lunares/'

In [5]:
lunas = esc.calendariolunar(url_,2016,2020)

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-

In [6]:
lunas.sample(3)

,fecha,moon,fecha_Date
12,30/03/2016,Cuarto Menguante,2016-03-30
197,26/12/2019,Luna Nueva,2019-12-26
72,16/06/2017,Cuarto Menguante,2017-06-16


In [8]:
lunas.to_csv('../data/cal_lunar.csv',index=False)

# Horas de luz

[web](https://www.epdata.es/horas-luz-provincias-2019/6841c77e-e946-420e-a0c4-cff4de1154f3#)

In [29]:
luz = pd.read_csv('../data/ambientales/horas_de_luz_por_provincias.csv',sep=';')
luz = luz[0:624].drop('Año',axis=1).rename({'Periodo':'mes','Parámetro':'Provincia','Salida y puesta del sol':'horas'},axis=1)
luz.horas = luz.horas.str.replace(',','.').astype('float')
luz.Provincia = luz.Provincia.str.strip()
luz.Provincia = luz.Provincia.str.replace('Islas Baleares','Baleares')

In [30]:
luz.sample(3)

,mes,Provincia,horas
447,Septiembre,Ourense,12.49
444,Septiembre,Málaga,12.42
440,Septiembre,Lleida,12.48


In [31]:
#coordenar nombre de provincias con geojson provincial
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

prov_cod = {}
for i in range(52):    
    prov_cod[provincias['features'][i]['properties']['name']] = provincias['features'][i]['properties']['cod_prov']

In [32]:
luz.Provincia.unique()

array(['Álava', 'Albacete', 'Alicante', 'Almería', 'Ávila', 'Badajoz',
       'Baleares', 'Barcelona', 'Burgos', 'Cáceres', 'Cádiz', 'Castellón',
       'Ciudad Real', 'Córdoba', 'A Coruña', 'Cuenca', 'Girona',
       'Granada', 'Guadalajara', 'Guipúzcoa', 'Huelva', 'Huesca', 'Jaén',
       'León', 'Lleida', 'La Rioja', 'Lugo', 'Madrid', 'Málaga', 'Murcia',
       'Navarra', 'Ourense', 'Asturias', 'Palencia', 'Las Palmas',
       'Pontevedra', 'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria',
       'Segovia', 'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia', 'Valladolid', 'Vizcaya', 'Zamora', 'Zaragoza', 'Ceuta',
       'Melilla'], dtype=object)

In [33]:
luz['cod'] = luz.Provincia.map(prov_cod)

In [34]:
luz.sample(3)

,mes,Provincia,horas,cod
574,Diciembre,Alicante,9.55,03
217,Mayo,Cáceres,14.35,10
179,Abril,León,13.38,24


In [35]:
luz.to_csv('../data/luzmesespr.csv',index=False)

# Provincias

In [36]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [37]:
cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [38]:
provincias_cod = pd.DataFrame([[key, cod_prov[key]] for key in cod_prov.keys()], columns=['Cod', 'Provincia'])

In [39]:
provincias_cod.to_csv('../data/codigos_provincia.csv',index=False)